In [14]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
import threading
import random
from skimage import io
from sklearn.cluster import KMeans



kernel = np.ones((5,5),np.uint8)
blnk = cv2.imread('blank.png', 0)
blnk = cv2.resize(blnk,(260,680))
centroidsAll = [[0,0,0,0]]


class camshift:
    int r,h,c,w
    ret = [] 
    hsv_roi = None
    track_window = None
    mask = None
    roi_hist = None
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
    coord = []
    global hsv, loadModel
    def create_csft(cx, cy):
        self.r, self.h, self.c, self.w = (cx-15), 30, (cy-15), 30 
        self.roi = frame[self.r:self.r+self.h, self.c:self.c+self.w]
        self.hsv_roi =  cv2.cvtColor(self.roi, cv2.COLOR_BGR2HSV)
        self.mask = cv2.inRange(self.hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
        self.roi_hist = cv2.calcHist([self.hsv_roi],[0],self.mask,[180],[0,180])
        cv2.normalize(self.roi_hist,self.roi_hist,0,255,cv2.NORM_MINMAX)
    def comp():
        hist2 = 
        if cv.CompareHist(roi_hist, hist2, method)<    
            return True
        else:
            return False
    def track_csft():
        dst = cv2.calcBackProject([hsv],[0],self.roi_hist,[0,180],1)
        self.ret, self.track_window = cv2.CamShift(dst, self.track_window, term_crit)
    def preTrackPred_csft():
        loadModel = None
    def plot():
        pts = cv2.boxPoints(self.ret)
        pts = np.int0(pts)
        return [pts]
#         img2 = cv2.polylines(frame,[pts],True, 255,2)
#         cv2.imshow('img2',img2)        
        
    
def sift_knn(img1,img2,part):
    
    global centroidsAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    for i, (m1,m2) in enumerate(matches):
        if m1.distance < 0.9 * m2.distance:
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             col = list(img2[int(pt2[1])-1 : int(pt2[1])+1,int(pt2[0])-1 : int(pt2[0])+1,:].flatten())
            info = [10*pt2[0], 10*pt2[1], 50*(pt2[1]-pt1[1]), 50*(pt2[0]-pt1[0])]#+col
#             print("yeah:",img2[int(pt2[0])-1 : int(pt2[0])+1,int(pt2[1])-1 : int(pt2[1])+1,:].flatten())
            infolist.append(info)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)



    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 2)
    res = cv2.drawMatchesKnn(img2,kp1,img2,kp2,matches,None,**draw_params)    
    
    infolist = np.array(infolist).reshape(-1,4)
    nc = min(infolist.shape[0],10)
    
    kmeans = KMeans(n_clusters = nc, n_init=10, max_iter=100)
    kmeans.fit(infolist)
    centroid = kmeans.cluster_centers_
    centroidsAll += list(centroid)
    for cnt in centroid:
        cv2.circle(img2,(int(cnt[0]/10),int(cnt[1]/10)),5,(0,0,255),-1)
    
    
    
    
        
def obstacleNcounters():
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    
    print("start")
    global centroidsAll
    vrec = cv2.VideoCapture('v4.mp4',0)
    k = 300*8
    prev1 = []
    shape = [260,680]
    cfstObj = []
    
    global loaded_model
    with open("pathInertia.json", "r") as json_file:
        loaded_model_json = json_file.read()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights("pathInertia.h5")
        print("Model loaded from disk")
###
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
###    
    while True:
        
        start_time = time.time()
        centroidsAll = []
        ret, scr = vrec.read()
        scr1 = scr
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (shape[1],shape[0]))#1360,520
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(shape[1],shape[0]))
    
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray,50,100)
        canny = blank - canny
        
        scr1 = canny[0:shape[0]//2, 0:shape[1]//3]
        scr2 = canny[0:shape[0]//2,shape[1]//3:2*shape[1]//3]
        scr3 = canny[0:shape[0]//2,2*shape[1]//3:shape[1]]
        scr4 = canny[shape[0]//2:shape[0], 0:shape[1]//3]
        scr5 = canny[shape[0]//2:shape[0],shape[1]//3:2*shape[1]//3]
        scr6 = canny[shape[0]//2:shape[0],2*shape[1]//3:shape[1]]
        if prev1 == [] :
            prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1 = threading.Thread(target = sift_knn, args=(prev1,scr1,0))#,0,0))
        th2 = threading.Thread(target = sift_knn, args=(prev2,scr2,1))#,0,shape[1]//3))
        th3 = threading.Thread(target = sift_knn, args=(prev3,scr3,2))#,0,2*shape[1]//3))
        th4 = threading.Thread(target = sift_knn, args=(prev4,scr4,3))#,shape[0]//2,0))
        th5 = threading.Thread(target = sift_knn, args=(prev5,scr5,4))#,shape[0]//2,shape[1]//3))
        th6 = threading.Thread(target = sift_knn, args=(prev6,scr6,5))#,shape[0]//2,2*shape[1]//3))
        th1.start()
        th2.start()
        th3.start()
        th4.start()
        th5.start()
        th6.start()
        
        frame = cv2.GaussianBlur(scr, (3, 3), 0)
        scr = cv2.addWeighted(scr,10,frame,-9,0)
        cv2.imshow('scred',scr)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        Cy = cv2.Canny(gray,50,100)
        Cy = blank - Cy
        im2, contours, hierarchy = cv2.findContours(Cy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(Cy,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,3)
        cv2.imshow("blk1",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        scr = cv2.drawContours(scr, contours1, -1, 0, 1)
        cv2.imshow("blk2",scr)
        
        for obj in cfstObj:
            obj.track_csft()
            obj.preTrackPred_csft()
            hist1 = 
            cv.CompareHist(hist1, hist2, method)
        
        prev1, prev2, prev3, prev4, prev5, prev6 = scr1, scr2, scr3, scr4, scr5, scr6
        th1.join()
        th2.join()
        th3.join()
        th4.join()
        th5.join()
        th6.join()
        cv2.imshow('print',canny)
        for cnt in centroidsAll:
            point = Point(cnt[0]/10,cnt[1]/10)
            for obj in cfstObj:
                polygon = Polygon(obj.ret)
                if not polygon.contains(point):
                    tempobj = camshift()
                    tempobj.create_csft(cnt[0]/10,cnt[1]/10)
                    tempobj.track_csft()
                    tempobj.preTrackPred_csft()
                    cfstObj.append(temp)
                    break
                    
                
#         kmeanC = KMeans(n_clusters = 10, n_init=10, max_iter=100)
#         kmeanC.fit(centroidsAll)
#         centroid = kmeanC.cluster_centers_
#         for cnt in centroid:
#              cv2.circle(canny,(int(cnt[0]/2),int(cnt[1]/2)),10,(0,0,255),-1)
        cv2.imshow('print2',canny)        
#         for  

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()

Test server listening on port 4000

start
running
(60, 4)
FPS:  1.422341348711864


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:119: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


(60, 4)
FPS:  2.2735277345943614
(60, 4)
FPS:  2.438190196442035
(60, 4)
FPS:  2.2950834383483163
(60, 4)
FPS:  2.4683861726250274
(60, 4)
FPS:  1.98486801631695
(60, 4)
FPS:  2.241583980623515
(60, 4)
FPS:  2.6864899971689495
(60, 4)
FPS:  2.5366094388415417
(60, 4)
FPS:  2.601497892716148
(60, 4)
FPS:  2.43110287792969
(60, 4)
FPS:  2.5929451533553456
(60, 4)
FPS:  2.5923794266275055
(60, 4)
FPS:  2.5237883204124416
(60, 4)
FPS:  2.841374843935554
(60, 4)
FPS:  2.722467360670273
(60, 4)
FPS:  2.680345620662088
(60, 4)
FPS:  2.5985226542660325
(60, 4)
FPS:  2.6870510694892538
(60, 4)
FPS:  2.886643120046469
(60, 4)
FPS:  2.9285394601516392
(60, 4)
FPS:  2.822848474365696
(60, 4)
FPS:  2.6513103249484504
(60, 4)
FPS:  2.742664036236637
(60, 4)
FPS:  2.4118418949239984
(60, 4)
FPS:  2.54942046524463
(60, 4)
FPS:  2.5608252659548487
(60, 4)
FPS:  2.463886761063777
(60, 4)
FPS:  2.4716327917231644
(60, 4)
FPS:  2.6157115916143647
(60, 4)
FPS:  2.5436411457805845
(60, 4)
FPS:  2.5032789742

In [24]:

cnt = np.array([[0,0], [1,1], [1,0],[0,1]])
bbox = cv2.minAreaRect(cnt)
pts = cv2.boxPoints(bbox)
print(pts)

[[  1.00000000e+00   1.00000000e+00]
 [ -3.06161713e-17   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00]]


In [15]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

True


In [ ]:
def sift_knn(img1,img2):
    global infoAll
    infolist = []
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    matchesMask = [[0,0] for i in range(len(matches))]
          
    # ratio test as per Lowe's paper
    j=0
    k=0    
    for i, (m1,m2) in enumerate(matches):
        k+=1
        if m1.distance < 0.7 * m2.distance:
            j+=1
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
            info = [lambda pt1, pt2: [10*pt2[0], 10*pt2[1], 10*(pt2[1]-pt1[1]), 10*(pt2[0]-pt1[0]), img2[pt2[0]-5 : pt2[0]+5,pt2[1]-5 : pt2[1]+5].flatten()]]
            infolist.append(info)
#             print(np.array(info).shape)
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img2, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)
        
#     print(j)
#     print(k)
    # Draw match in blue, error in red
    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 0)
    res = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
#     cv2.imshow("res",res)
    try:
        infoAll.append(infolist)
    except:
        time.sleep(0.001*random.randint(1,9))
        infoAll.append(infolist)
        

In [ ]:
import numpy as np
    2 import cv2
    3 
    4 cap = cv2.VideoCapture('slow.flv')
    5 
    6 # take first frame of the video
    7 ret,frame = cap.read()
    8 
    9 # setup initial location of window
   10 r,h,c,w = 250,90,400,125  # simply hardcoded the values
   11 track_window = (c,r,w,h)
   12 
   13 # set up the ROI for tracking
   14 roi = frame[r:r+h, c:c+w]
   15 hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
   16 mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
   17 roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
   18 cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
   19 
   20 # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
   21 term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
   22 
   23 while(1):
   24     ret ,frame = cap.read()
   25 
   26     if ret == True:
   27         hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
   28         dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
   29 
   30         # apply meanshift to get the new location
   31         ret, track_window = cv2.CamShift(dst, track_window, term_crit)
   32 
   33         # Draw it on image
   34         pts = cv2.boxPoints(ret)
   35         pts = np.int0(pts)
   36         img2 = cv2.polylines(frame,[pts],True, 255,2)
   37         cv2.imshow('img2',img2)
   38 
   39         k = cv2.waitKey(60) & 0xff
   40         if k == 27:
   41             break
   42         else:
   43             cv2.imwrite(chr(k)+".jpg",img2)
   44 
   45     else:
   46         break
   47 
   48 cv2.destroyAllWindows()
 cap.release()

In [20]:
import cv2
import numpy as np

# A class named "CamShift" is defined which consists of variables and methods used to implement the CamShift algorithm for tracking objects:
class CamShift(): 

    # Function to initialize the variables and reading video frames: 
    def __init__(cls): 
        cls.FLAG_track = 0

        # Initializing Region of interest :
        cls.region_of_interest = None
        # Initializing coordinates of mouse click :
        cls.rect_coord = None

        # Creating a VideoCapture object in order to capture a video from camera:
        cls.cap = cv2.VideoCapture(0)
        
        # Initializing the current frame 'video_frame' of the video to process frame by frame :
        ret, cls.video_frame = cls.cap.read()

        # Creating a window with name 'Tracking' :
        cv2.namedWindow('Tracking', 1)

        # Setting mouse handler for the window 'Tracking' by calling the function 'MouseClick' :
        cv2.setMouseCallback('Tracking', cls.MouseClick, None)

    # The function below is used to select our region of interest in the video frame: 
    ## Function to define what happens when we click a mouse button :
    def MouseClick(cls, event, x, y, flags, param):
        
        # Event for which left mouse is being clicked, or released (giving click's x & y coordinates):
        if event == cv2.EVENT_LBUTTONDOWN:

            # Storing x and y coordiantes of mouse click to the points of region of interest to be used to track the object :
            cls.rect_coord = (x, y) 
        
            cls.FLAG_track = 0
            return

        if cls.rect_coord:
            if flags & cv2.EVENT_FLAG_LBUTTON:
                # Getting the dimesions using shape of frame and storing it in variables s1 and s2:
                s1, s2 = cls.video_frame.shape[:2]

                # Taking the rectangle coordinates into variables a and b :
                a,b = cls.rect_coord

                # Setting the four co-ordinates of the rectangle :
                a1,b1 = np.maximum(0, np.minimum([a,b], [x, y]))
                a2,b2 = np.minimum([s2, s1], np.maximum([a,b], [x, y]))
                cls.region_of_interest = None
                if a2-a1 > 0 and b2-b1 > 0:
                    # Setting the region of interest window with coordinates:
                    cls.region_of_interest = (a1,b1,a2,b2) 
            else:
                cls.rect_coord = None 
                if cls.region_of_interest is not None: 
                    cls.FLAG_track = 1 

    def hue(cls):
        b = 24
        # Taking the histogram bins according to the shape of image's histogram: 
        bins = cls.histogram.shape[0]
        
        # Creating a histogram by initializing a 3d array for the given size with all array values initially equal to zero:
        img = np.zeros((256, bins*b, 3), np.uint8)
        
        for j in range(bins):

            # For all histogram bins, taking the integer values for selected channels :
            s1 = int(cls.histogram[j])

            # Choosing the region shape to be tracked representing mouse's click location:
            cv2.rectangle(img, (j*(b+2), 255), ((j+1)*(b-2), 255-s1), (int(180*(j/bins)), 255, 255), -1)
        
        # Coverting image from HSV color space to RGB color space in order to show the histogram image in RGB format:
        img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR) 


    def main(cls): 
        while True: # Loop used for processing all the frames

            # Taking each frame of the video:
            ret, cls.video_frame = cls.cap.read()

            # Copying the frame of the video ('new_w' is an image type):
            new_w = cls.video_frame.copy() 

            # Converting the tracked window (of region of interest) of the video to HSV color space :
            region_of_interest_hsv = cv2.cvtColor(cls.video_frame, cv2.COLOR_BGR2HSV)

            # Thresholding the HSV image to get the R,G,B colors :
            mask = cv2.inRange(region_of_interest_hsv, np.array((0., 30., 32.)), np.array((180., 255., 255.)))

            if cls.region_of_interest:
                a1,b1, a2,b2 = cls.region_of_interest
                
                # Estimated size of the window in between which object is tracked : 
                cls.window = (a1,b1, (a2-a1), (b2-b1))

                # Creating an HSV image with the given coordinates:
                image_hsv = region_of_interest_hsv[b1:b2, a1:a2]

                # Creating mask image in order to find histogram of any specific image region:  
                image_mask = mask[b1:b2, a1:a2]

                # Calculating the HSV histogram of arrays (with 16 values to represent histogram and range of HSV color space[0,180]) :
                histogram = cv2.calcHist( [image_hsv], [0], image_mask, [16], [0, 180] )

                # Normalizing the histogram with parameter as input and output histogram, gray scale vlaues range and 
                cv2.normalize(histogram, histogram, 0, 255, cv2.NORM_MINMAX)
                cls.histogram = histogram.reshape(-1)
                
                # Calling the 'hue' function to choose the region shape to be tracked based on histogram bins:
                cls.hue()
                # Setting the new region window for frame:
                region_new_w = new_w[b1:b2, a1:a2]

                # Bitwise NOT operation used to choose the shape of the object to be tracked as rectangle:
                cv2.bitwise_not(region_new_w, region_new_w)

            if cls.FLAG_track == 1:
                cls.region_of_interest = None

                # Calculate the Histogram Back Projection:
                ## Finding the corresponding histogram bin for different values collected from image channels :
                ### Calculating the probability values of each element with respect to the histogram's probability distribution :
                probability = cv2.calcBackProject([region_of_interest_hsv], [0], cls.histogram, [0, 180], 1)
#                 probability &= mask
                print(probability)
                # Termination Condition for the algorithm specifying the minimum movement by 1 pixel along region of interest & a maximum of 5 iterations:
                terminate = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1 )
                
                # CamSHift function giving us the new position of Region of interest and size, position of our desired object to be tracked:
                track, cls.window = cv2.CamShift(probability, cls.window, terminate) 

                # Setting the ellipse in order to define the shape of the tracked object by using the new position of region of interest :
                cv2.ellipse(new_w, track, (255, 0, 255), 3) 
                 
            # Showing the tracking window by drawing new position of the region of interest on the image:
            cv2.imshow('Tracking', new_w)

            ch = 0xFF & cv2.waitKey(5)
            if ch == 27:
                break

        # Closing any open windows:
        cv2.destroyAllWindows()

# Calling the 'main' function of the class using the class object:
CamShift().main() 

[[3 3 0 ..., 0 0 0]
 [3 3 0 ..., 0 0 0]
 [3 3 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 1 1]
 [0 0 0 ..., 0 1 1]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 0 1 1]]
[[  0   0  35 ...,   0   0   0]
 [  3   3 255 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [  1   1   1 ...,   1   1   1]
 [  1   1   1 ...,   1   1   1]
 [  1   1   1 ...,   1   1   1]]
[[  3   3   3 ...,   1   0   0]
 [  3   3   3 ...,   1   0   0]
 [ 35  35 255 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   1   1   1]
 [  0   0   0 ...,   1   

[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 0 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 0 1 1]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[1 1 0 ..., 1 0 0]
 [1 1 0 ..., 1 0 0]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 

[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 0 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[1 1 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 1 0 ..., 1 1 1]
 [1 1 0 ..., 1 1 1]
 [1 1 0 ..., 1 1 1]]
[[1 1 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[1 1 0 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [0 0 0 ..., 1 1 

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 1 1]
 [0 0 0 ..., 0 1 1]
 [0 0 0 ..., 0 1 1]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 

[[1 1 1 ..., 0 1 1]
 [1 1 1 ..., 0 1 1]
 [1 1 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 1 0 0]
 [1 1 1 ..., 1 0 0]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 1 ..., 1 1 

...

[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 3 3]
 [0 0 0 ..., 0 0 0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]]
[[  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [255 255 255 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 3 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[  0 255   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  1   1   1 ...,   0   0   0]
 [  1   1   1 ...,   0   0   0]]
[[3 3 0 ..., 0 0 0]
 [3 3 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 

[[  0   0   1 ...,   1   1   1]
 [  0   0   1 ...,   1   1   1]
 [  0   0   0 ...,   1   1   1]
 ..., 
 [  1   1   1 ...,   0   0   0]
 [  1   1   1 ...,  35   0 255]
 [  1   1   1 ...,  35   0 255]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 1 1]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 [0 1 1 ..., 0 0 0]]
[[0 0 1 ..., 1 1 1]
 [0 0 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 1 ..., 1 1 1]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 1 1]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 0 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 0 1 1]
 [1 1 1 ..., 0 1 1]]
[[1 1 0 ..., 1 1 1]
 [1 1 1 ...,

[[0 0 1 ..., 1 1 1]
 [0 0 1 ..., 1 1 1]
 [0 0 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 [1 1 1 ..., 1 0 0]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 3 3]
 [1 1 1 ..., 0 3 3]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 3 0 0]
 [1 1 1 ..., 3 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 3 0 0]
 [1 1 1 ..., 3 0 0]]
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 

[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [0 0 0 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]]
[[  0   0   1 ...,   1   1   1]
 [  1   0   1 ...,   1   1   1]
 [  1   1   1 ...,   1   1   1]
 ..., 
 [  1   1   1 ..., 255   1   1]
 [  1   1   1 ...,   0   0   0]
 [  1   1   1 ...,   0   0   0]]
[[0 0 1 ..., 1 1 1]
 [0 0 1 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 ..., 
 [1 1 0 ..., 1 0 0]
 [1 1 0 ..., 1 0 0]
 [1 1 0 ..., 1 0 0]]
[[1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[1 1 1 ..., 1 0 0]
 [1 1 1 ..., 1 0 0]
 [0 0 0 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[  1   1   1 ...,   0 255 255]
 [  1   1   1 ...,   0 255 255]
 [  1   1   1 ...,   3 255 255]
 ..., 
 [  1   1   1 ...,   0   0   0]
 [  1   1   1 ...,   0   

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 1 1]
 [0 0 0 ..., 0 1 1]
 [0 0 0 ..., 0 1 0]]
[[0 0 0 ..., 1 1 1]
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [1 1 0 ..., 1 1 1]
 [1 1 0 ..., 1 1 1]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 1 1 ..., 0 1 1]
 [1 1 1 ..., 0 0 0]
 [1 0 1 ..., 0 0 0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 1 1]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0 255 255 ...,   0   0   0]
 ..., 
 